<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
<h1 style="display:none;">Test</h1>
# Introduction to Databases: SELECT Continued, Views, Constraints, HW 2


## Some Helper Code Used in Cells in the Notebook


In [55]:
# We need a module (library) that can communicate with the
# database server.
import pymysql.cursors
import pandas as pd
import json


# The database server is running somewhere in the network.
# I must specify the IP address (HW server) and port number
# (connection that SW server is listening on)
# Also, I do not want to allow anyone to access the database
# and different people have different permissions. So, the
# client must log on.
config = {
  'user': 'dbuser',
  'password': 'dbuser',
  'host': '10.0.1.4',
  'database': 'lahman2016',
  'raise_on_warnings': True,
  'charset' : 'utf8'
}

# Connect
cnx = pymysql.connect(host='localhost',
                             user='dbuser',
                             password='dbuser',
                             db='lahman2016',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

def run_query(q):
    print("Execution query = \n",q)
    with cnx.cursor() as cursor:
        # Create a new record
        cursor.execute(q)

    df_mysql = pd.read_sql(q,cnx)
    return df_mysql


def print_result(msg, pf):
    print(msg)
    print(pf)
    
def run_and_print_q(msg,q):
    r = run_query(q)
    print("\n",msg,"\n",r)
    
    
def run_query_to_d(q):
     with cnx.cursor() as cursor:
        # Create a new record
        cursor.execute(q)
        
     r = cursor.fetchall()
     return r


## Questions, Comments, Discussion

### Questions or Comments?

### Collation and Comparison

__Version 1 of the Piazza Question__

| <img src="../images/L5_collation.jpeg"> | 
|:--:| 
|__Collation__ |

__Version 2 of the Piazza Question__


| <img src="../images/L5_hash_index_column.jpeg"> | 
|:--: | 
| __Comparison Operators__ | 

- The basic pattern for binary operators like $+, -, \geq, \leq$ is $operand1\ operator\ operand2$, e.g.
    - $x \leq 21$
    - $y + x$
    
    
- The exact meaning of the _operator_ is _context sensitive_ relative to the operands. In many cases, the languages "do the right thing."


In [43]:
x = "Cat's" + " and " + "Dogs"
y = 21 / 4
print("x = ", x)
print("y = ", y)
print("Notice that 21 / 4 resulted in a float even though both operands were integers.")
print("\nThe context sensitive nature works for more complex types.")
l1 = ["red", "orange", "green", "blue"]
l2 = ["yellow", "indigo", "violet"]
l3 = l1 + l2
print("l1 = ", l1)
print("l2 = ", l2)
print("l1 + l2 = ", l3)


x =  Cat's and Dogs
y =  5.25
Notice that 21 / 4 resulted in a float even though both operands were integers.

The context sensitive nature works for more complex types.
l1 =  ['red', 'orange', 'green', 'blue']
l2 =  ['yellow', 'indigo', 'violet']
l1 + l2 =  ['red', 'orange', 'green', 'blue', 'yellow', 'indigo', 'violet']


- Human languages and supporting computer _character sets_ have a _sort order._
    - English 'a' is before 'q'
    - Swedish collation rules determine if Å comes before or after A.
    
    
- Programming languages and databases have complex functions for defining exactly what is in strings and the sort order.

In [35]:
animal_1 = "Aardvark"
animal_2 = "Zebra"

if animal_1 < animal_2:
    print(animal_1, " is LT ", animal_2)

Aardvark  is LT  Zebra


In [45]:
q = "SELECT playerID, nameLast, nameFirst FROM lahman2016.Master WHERE nameLast <= 'ab';"
with cnx.cursor() as cursor:
        # Create a new record
        cursor.execute(q)

df_mysql = pd.read_sql(q,cnx)
print("Result set = \n", df_mysql)

Result set = 
     playerID nameLast nameFirst
0  aardsda01  Aardsma     David
1  aaronha01    Aaron      Hank
2  aaronto01    Aaron    Tommie
3   aasedo01     Aase       Don


- The core SQL/MySQL column types are

| Data type            | Description                                                                                                                                                                                  |
|----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| CHARACTER(n)         | Character string. Fixed-length n                                                                                                                                                             |
| VARCHAR(n) or        | Character string. Variable length. Maximum length n                                                                                                                                          |
| CHARACTER VARYING(n) | Character string. Variable length. Maximum length n                                                                                                                                          |
| BINARY(n)            | Binary string. Fixed-length n                                                                                                                                                                |
| BOOLEAN              | Stores TRUE or FALSE values                                                                                                                                                                  |
| VARBINARY(n) or      | Binary string. Variable length. Maximum length n                                                                                                                                             |
| BINARY VARYING(n)    | Binary string. Variable length. Maximum length n                                                                                                                                             |
| INTEGER(p)           | Integer numerical (no decimal). Precision p                                                                                                                                                  |
| SMALLINT             | Integer numerical (no decimal). Precision 5                                                                                                                                                  |
| INTEGER              | Integer numerical (no decimal). Precision 10                                                                                                                                                 |
| BIGINT               | Integer numerical (no decimal). Precision 19                                                                                                                                                 |
| DECIMAL(p,s)         | Exact numerical, precision p, scale s. Example: decimal(5,2) is a number that has 3 digits before the decimal and 2 digits after the decimal                                                 |
| NUMERIC(p,s)         | Exact numerical, precision p, scale s. (Same as DECIMAL)                                                                                                                                     |
| FLOAT(p)             | Approximate numerical, mantissa precision p. A floating number in base 10 exponential notation. The size argument for this type consists of a single number specifying the minimum precision |
| REAL                 | Approximate numerical, mantissa precision 7                                                                                                                                                  |
| FLOAT                | Approximate numerical, mantissa precision 16                                                                                                                                                 |
| DOUBLE PRECISION     | Approximate numerical, mantissa precision 16                                                                                                                                                 |
| DATE                 | Stores year, month, and day values                                                                                                                                                           |
| TIME                 | Stores hour, minute, and second values                                                                                                                                                       |
| TIMESTAMP            | Stores year, month, day, hour, minute, and second values                                                                                                                                     |
| INTERVAL             | Composed of a number of integer fields, representing a period of time, depending on the type of interval                                                                                     |
| ARRAY                | A set-length and ordered collection of elements                                                                                                                                              |
| MULTISET             | A variable-length and unordered collection of elements                                                                                                                                       |
| XML                  | Stores XML data                                                                                                                                                                              |
| | <b>SQL Column Types<b> |

- MySQL and other database engines
    - Support extensions.
    - Do not support all of the types above, e.g. XML, MULTISET, ...
    

- All of the core types support ordering: strings, numbers, date/time.


- Just because a column supports ordering/collation, does not mean that the web application would support the function or that the database administrator would allow it.
    - Sometimes columns are generated [UUIDs](https://en.wikipedia.org/wiki/Universally_unique_identifier) or hashed (e.g. passwords).
    - They are strings, and are orderable, but due to the use case, ordering is not meaningful.
    - I can envision scenarios in which queries would use LIKE for UNIs but not ordering, for example. The same applies for playerID.
    - For now, do not worry about this detail and view all strings to support collation.

## Integrity Constraints

### Motivation

- I plan to cover INSERT, DELETE and UPDATE next week.


- Integrity constraints apply to INSERT, UPDATE and DELETE.


- HW2 requires use of some constraints, specifically primary and foreign keys $\Rightarrow$ I need to cover a little bit now.

### Overview

"Data integrity is normally enforced in a database system by a series of integrity constraints or rules. Three types of integrity constraints are an inherent part of the relational data model: _entity integrity, referential integrity and domain integrity:_
- __Entity integrity__ concerns the concept of a primary key. Entity integrity is an integrity rule which states that every table must have a primary key and that the column or columns chosen to be the primary key should be unique and not null.


- __Referential integrity__ concerns the concept of a foreign key. The referential integrity rule states that any foreign-key value can only be in one of two states. The usual state of affairs is that the foreign-key value refers to a primary key value of some table in the database. Occasionally, and this will depend on the rules of the data owner, a foreign-key value can be null. In this case, we are explicitly saying that either there is no relationship between the objects represented in the database or that this relationship is unknown.


- __Domain integrity__ specifies that all columns in a relational database must be declared upon a defined domain. The primary unit of data in the relational data model is the data item. Such data items are said to be <span style="color: red;">non-decomposable or atomic.</span> A domain is a set of values of the same type. Domains are therefore pools of values from which actual values appearing in the columns of a table are drawn.


- __User-defined__ integrity refers to a set of rules specified by a user, which do not belong to the entity, domain and referential integrity categories."

"Constraints are used to limit the type of data that can go into a table. This ensures the accuracy and reliability of the data in the table. If there is any violation between the constraint and the data action, the action is aborted." (https://www.w3schools.com/sql/sql_constraints.asp)

__Note:__
- https://www.w3schools.com/sql/default.asp is a pretty good tutorial on SQL if you need to review or play with SQL.
- I will cover a lot of SQL, and the CAs and I will always help and answer questions. 


__Common Integrity Constrains:__
- NOT NULL - Ensures that a column cannot have a NULL value
- UNIQUE - Ensures that all values in a column are different
- PRIMARY KEY - A combination of a NOT NULL and UNIQUE. Uniquely identifies each row in a table
- FOREIGN KEY - Uniquely identifies a row/record in another table
- CHECK - Ensures that all values in a column satisfies a specific condition
- DEFAULT - Sets a default value for a column when no value is specified
- INDEX - Used to create and retrieve data from the database very quickly

__Note:__
- The details of supported constraints vary between database management systems.
- MySQL does not support CHECK and uses [Triggers](https://en.wikipedia.org/wiki/Database_trigger), which we will cover.
- CHECK/Triggers are the primary mechanism for implementing user-defined integrity rules.



### Why Constraints

| <img src="http://slideplayer.com/slide/4775270/15/images/4/Client+-+Server+Databases.jpg"> | 
|:--:| 
|[__Database Client-Server__](http://slideplayer.com/slide/4775270/15/images/4/Client+-+Server+Databases.jpg) |


- The database server is "on the network." Any application with proper credentials can connect to the database.


- There could be many, many applications written by many, many teams.


- Assuming that every team will get all the constraints correct in their application is risky.


- Changing constraints as the business/data model evolves requires finding and mondifying many programs.


- Integrity constraints defined as part of the data model and enforced by the database management system for all clients is a fundamental function of the relational model.


### MySQL Constraints Overview

| <img src="../images/L5_mysql_constraints.jpeg"> | 
|:--:| 
|__MySQL Workbench and Constraints__ |

__Simple Demo__
- Column constraints.
- Identifying and defining a primary key.
- Defining foreign keys.

## Simple SQL SELECT and PROJECT Examples

### Motivation

- The best way to learn SQL is practice and examples.


- We will go through some simple, interesting, educational examples.

### LIKE Operator

- Syntax is "column LIKE pattern"


- Pattern is a string that may contain wildcards
    - % : The percent sign represents zero, one, or multiple characters
    - _ : The underscore represents a single character


In [3]:
q = "SELECT nameLast, nameFirst FROM Master WHERE " + \
    "nameFirst LIKE 'T_d' and nameLast LIKE 'C%'"
run_and_print_q("Query with wildcard", q)

Execution query = 
 SELECT nameLast, nameFirst FROM Master WHERE nameFirst LIKE 'T_d' and nameLast LIKE 'C%'

 Query with wildcard 
   nameLast nameFirst
0   Cather       Ted
1  Cieslak       Ted
2      Cox       Ted


### Operators on Columns

- You can perform operations on columns in the project and where clauses.


- There is a large set of [standard SQL](https://www.w3schools.com/sql/sql_operators.asp) operators/functions. [MySQL functions and operators](https://dev.mysql.com/doc/refman/5.7/en/func-op-summary-ref.html) as a superset.


- The functions operators manipulate numbers, strings, dates, etc.

In [4]:
q = "SELECT nameLast, nameGiven, concat(nameGiven,' ',nameLast) AS full_name FROM Master LIMIT 10"
run_and_print_q("\nConcatenate nameGive and nameLast to full name\n", q)
print("\n\n")

q_on = "SELECT " + \
	"playerID, " + "AB as at_bats, " + \
    "ROUND((H + BB + IBB + HBP)/(AB + BB + IBB + HBP),3)*1000 AS on_base_percent, " + \
    "ROUND((H / AB),3)*1000 AS batting_everage " + \
    "FROM Batting WHERE  " + \
		"AB  > 200  AND ROUND((H / AB),3)*1000  > 300 LIMIT 10;"
run_and_print_q("\nCompute on_base percentage and batting average.\n", q_on)

Execution query = 
 SELECT nameLast, nameGiven, concat(nameGiven,' ',nameLast) AS full_name FROM Master LIMIT 10

 
Concatenate nameGive and nameLast to full name
 
       nameLast         nameGiven                 full_name
0      Aardsma       David Allan       David Allan Aardsma
1        Aaron       Henry Louis         Henry Louis Aaron
2        Aaron        Tommie Lee          Tommie Lee Aaron
3         Aase    Donald William       Donald William Aase
4         Abad     Fausto Andres        Fausto Andres Abad
5         Abad  Fernando Antonio     Fernando Antonio Abad
6       Abadie           John W.            John W. Abadie
7  Abbaticchio      Edward James  Edward James Abbaticchio
8        Abbey         Bert Wood           Bert Wood Abbey
9        Abbey        Charles S.          Charles S. Abbey



Execution query = 
 SELECT playerID, AB as at_bats, ROUND((H + BB + IBB + HBP)/(AB + BB + IBB + HBP),3)*1000 AS on_base_percent, ROUND((H / AB),3)*1000 AS batting_everage FROM Battin

### DISTINCT

- There may be duplicate tuples in a query result.


- SELECT DISTINCT removes duplicates.

In [5]:
q = "select nameLast FROM Master WHERE nameLast LIKE 'Willi%'"
run_and_print_q("Last names like Willi%",q)

Execution query = 
 select nameLast FROM Master WHERE nameLast LIKE 'Willi%'

 Last names like Willi% 
       nameLast
0     Williams
1     Williams
2     Williams
3     Williams
4     Williams
5     Williams
6     Williams
7     Williams
8     Williams
9     Williams
10    Williams
11    Williams
12    Williams
13    Williams
14    Williams
15    Williams
16    Williams
17    Williams
18    Williams
19    Williams
20    Williams
21    Williams
22    Williams
23    Williams
24    Williams
25    Williams
26    Williams
27    Williams
28    Williams
29    Williams
..         ...
69    Williams
70    Williams
71    Williams
72    Williams
73    Williams
74    Williams
75    Williams
76    Williams
77    Williams
78  Williamson
79  Williamson
80  Williamson
81  Williamson
82  Williamson
83  Williamson
84  Williamson
85   Willigrod
86  Willingham
87  Willingham
88      Willis
89      Willis
90      Willis
91      Willis
92      Willis
93      Willis
94      Willis
95      Willis
96      Wil

In [6]:
q = "select DISTINCT nameLast FROM Master WHERE nameLast LIKE 'Willi%'"
run_and_print_q("Distinct last names like Willi%",q)

Execution query = 
 select DISTINCT nameLast FROM Master WHERE nameLast LIKE 'Willi%'

 Distinct last names like Willi% 
      nameLast
0    Williams
1  Williamson
2   Willigrod
3  Willingham
4      Willis
5     Willits


## ORDER BY

- ORDER BY c1, c2, c3 orders the result table by c1, then c2, etc.

- You can order ASC or DESC

In [7]:
q_on2 = "SELECT " + \
	"playerID, " + "AB as at_bats, " + \
    "ROUND((H + BB + IBB + HBP)/(AB + BB + IBB + HBP),3)*1000 AS on_base_percent, " + \
    "ROUND((H / AB),3)*1000 AS batting_everage " + \
    "FROM Batting WHERE  " + \
		"AB  > 200  AND ROUND((H / AB),3)*1000  > 300 ORDER BY on_base_percent DESC LIMIT 10;"
run_and_print_q("\n10 highest season on_base percentages with ABs and batting average.\n", q_on2)

Execution query = 
 SELECT playerID, AB as at_bats, ROUND((H + BB + IBB + HBP)/(AB + BB + IBB + HBP),3)*1000 AS on_base_percent, ROUND((H / AB),3)*1000 AS batting_everage FROM Batting WHERE  AB  > 200  AND ROUND((H / AB),3)*1000  > 300 ORDER BY on_base_percent DESC LIMIT 10;

 
10 highest season on_base percentages with ABs and batting average.
 
     playerID  at_bats  on_base_percent  batting_everage
0  bondsba01      373            676.0            362.0
1  bondsba01      403            625.0            370.0
2  bondsba01      390            578.0            341.0
3  willite01      420            555.0            388.0
4  willite01      456            553.0            406.0
5  mcgrajo01      399            547.0            391.0
6   ruthba01      522            545.0            393.0
7  bondsba01      476            541.0            328.0
8   ruthba01      457            533.0            376.0
9  mantlmi01      474            532.0            365.0


In [8]:
q_sort = "SELECT nameLast, nameFirst FROM Master ORDER by nameLast, nameFirst ASC LIMIT 20;"
run_and_print_q("\n10 Players sorted by last name, then first name \n", q_sort)

Execution query = 
 SELECT nameLast, nameFirst FROM Master ORDER by nameLast, nameFirst ASC LIMIT 20;

 
10 Players sorted by last name, then first name 
 
        nameLast nameFirst
0       Aardsma     David
1         Aaron      Hank
2         Aaron    Tommie
3          Aase       Don
4          Abad      Andy
5          Abad  Fernando
6        Abadie      John
7   Abbaticchio        Ed
8         Abbey      Bert
9         Abbey   Charlie
10       Abbott       Dan
11       Abbott      Fred
12       Abbott     Glenn
13       Abbott      Jeff
14       Abbott       Jim
15       Abbott      Kurt
16       Abbott      Kyle
17       Abbott       Ody
18       Abbott      Paul
19         Aber        Al


## GROUP BY

### Basic Concept

<img src="../images/L5_group_by.jpeg">

In [9]:
q = "SELECT teamID, franchID, sum(W) as total_wins, sum(L) as total_loses, " + \
    "+ sum(w)+sum(l) as total_games, sum(w)/(sum(w)+sum(l)) as win_percent " + \
    " FROM Teams GROUP BY teamID, franchID " + \
    " order by win_percent DESC LIMIT 10;"
run_and_print_q("Baseball team win, loss, percent ordered by percent", q)

Execution query = 
 SELECT teamID, franchID, sum(W) as total_wins, sum(L) as total_loses, + sum(w)+sum(l) as total_games, sum(w)/(sum(w)+sum(l)) as win_percent  FROM Teams GROUP BY teamID, franchID  order by win_percent DESC LIMIT 10;

 Baseball team win, loss, percent ordered by percent 
   teamID franchID  total_wins  total_loses  total_games  win_percent
0    SLU      SLM        94.0         19.0        113.0       0.8319
1    BS1      BNA       225.0         60.0        285.0       0.7895
2    BS2      BRS        93.0         42.0        135.0       0.6889
3    CH1      CNA        19.0          9.0         28.0       0.6786
4    MLU      MLU         8.0          4.0         12.0       0.6667
5    PH1      PNA       165.0         86.0        251.0       0.6574
6    CNU      COR        69.0         36.0        105.0       0.6571
7    SL4      STL       781.0        432.0       1213.0       0.6439
8    BSP      BRS        81.0         48.0        129.0       0.6279
9    HAR      HAR  

Explanation:

- SELECT a set of rows and columns that optionally match a WEHRE condition.


- GROUP BY teamID, franchID forms a set of tuples for each distinct set of teamID, franchID values. For example, all of the tuples with teamID = SLU and franchID = SLM.


- The query will not convert the set into a single tuples.


- This means that for any _column_ we must convert a column (set) of values into a single value.


- If all of the columns are the same, this is easy.


- For columns that are not the same, we must apply aggregation operations, e.g.
    - SUM
    - MAX
    - MIN
    - AVG


### Having

- SELECT ... FROM ... WHERE ... GROUP BY produces a table.


- We already used "our WHERE clause" to determine what to GROUP.


- What if we only want some of the computed tuples to be in the final set? Use HAVING



In [10]:
q = "SELECT teamID, franchID, sum(W) as total_wins, sum(L) as total_loses, " + \
    "+ sum(w)+sum(l) as total_games, sum(w)/(sum(w)+sum(l)) as win_percent " + \
    " FROM Teams GROUP BY teamID, franchID " + \
    " HAVING total_games >= 500 " + \
    " order by win_percent DESC LIMIT 10;"
run_and_print_q("Baseball team win, loss, percent ordered by percent with minimum 500 games", q)

Execution query = 
 SELECT teamID, franchID, sum(W) as total_wins, sum(L) as total_loses, + sum(w)+sum(l) as total_games, sum(w)/(sum(w)+sum(l)) as win_percent  FROM Teams GROUP BY teamID, franchID  HAVING total_games >= 500  order by win_percent DESC LIMIT 10;

 Baseball team win, loss, percent ordered by percent with minimum 500 games 
   teamID franchID  total_wins  total_loses  total_games  win_percent
0    SL4      STL       781.0        432.0       1213.0       0.6439
1    PRO      PRO       438.0        278.0        716.0       0.6117
2    BLN      BLO       644.0        447.0       1091.0       0.5903
3    CN2      CIN       549.0        396.0        945.0       0.5810
4    NYA      NYY     10084.0       7648.0      17732.0       0.5687
5    ML1      ATL      1146.0        890.0       2036.0       0.5629
6    NY1      SFG      6067.0       4898.0      10965.0       0.5533
7    LAN      LAD      5048.0       4353.0       9401.0       0.5370
8    BR3      LAD       410.0        3

### Comparing Columns with Columns

- Up until now, we have just happened to compare columns or computed values to literals, e.g.
    - playerID = 'napolmi01'
    - nameLast LIKE 'Will%'
    - sum(w) + sum(l) > 500
    
    
- Predicates can compare columns with columns and functions of columns.


In [11]:
q = "SELECT yearID, teamID, franchID, w, l FROM Teams WHERE w < 0.5*l and (w + l) > 100 LIMIT 10"
run_and_print_q("Really bad teams", q)

Execution query = 
 SELECT yearID, teamID, franchID, w, l FROM Teams WHERE w < 0.5*l and (w + l) > 100 LIMIT 10

 Really bad teams 
    yearID teamID franchID   w   l
0    1884    CL2      CBL  35  77
1    1884    DTN      DTN  28  84
2    1884    IN2      IHO  29  78
3    1884    PT1      PIT  30  78
4    1886    KCN      KCN  30  91
5    1886    WS8      WNL  28  92
6    1887    CL3      CLV  39  92
7    1887    IN3      IND  37  89
8    1887    NY4      NYP  44  89
9    1888    KC2      KCC  43  89


## OK, That Was Tedious and Raises Some Questions

### Why Did I Cover that Material

- Some of these capabilities will be helpful for HW2 and future data analysis projects
    - Player career totals for batting, appearances, etc. require GROUP BY.
    - User's are interested in querying on derived values, e.g. batting average, win percentage, etc.
    
    
- You can learn the nitty-gritty details by practice, experimentation, office hours, ...


- I want to make sure you understand the overall capabilities to help you get started on projects.


- We cannot use all of the capabilities in our small homeworks. There will be some questions on examples.


### Why Not Write Python (or other) Functions?

- Not everyone can or wants to write functions and programs, but want to perform _ad hoc_ queries on demand.

"An ad hoc query is a data search designed for an out of the ordinary circumstance or question—not one of the pre-programmed queries that many organizations run on a regular basis." (https://businessintelligence.com/dictionary/ad-hoc-query/)

_ad hoc query:- "A non-standard inquiry. An ad hoc query is created to obtain information as the need arises. Contrast with a query that is predefined and routinely processed."(https://www.pcmag.com/encyclopedia/term/37486/ad-hoc-query)


- A simple, focused language enables a broader community of people to interact with data, and enables simpler, more focused tools.

__Quick Site Demo__

<img src="../images/L5_quick_site.jpeg">


## Data Design and Views

Ramakrishnan and Gehrke, section 19.1

- Why isn't the player's name and other useful information in Appearances, Batting, etc.

<img src="../images/L5_views_1.jpeg">

- Why not store the player and team/franchise names in the Batting and other tables?


- Redundancy Problem:
    - Inefficient use of storage.
    - Anamolies
        - Update: Some one, somewhere will forget to update the replicas.
        - Insertion: We cannot insert new data without knowing all of the other places we need a copy.
        - Deletion: We may delete one instances, but leave other instances undeleted.
        
        
- Relational databases focus on _integrity,_ which implies avoiding redundancy.


- No problem. I can just get the data I want by  ...

```
SELECT 
	playerID, 
    (select nameLast from Master where Master.playerID = Batting.playerID) as last_name,
    (select nameFirst from Master where Master.playerID = Batting.playerID) as first_name,
    (select franchName from TeamsFranchises WHERE
		TeamsFranchises.franchID = 
                (select franchID from Teams where Teams.teamID = 
                    Batting.teamID and Teams.yearID = Batting.yearID))
                        as franchise_name,
    teamID, yearID, round((h/ab),3) as batting_avg FROM
    lahman2016.batting where ab > 100
    order by batting_avg DESC;
```

- And then what if I want to query THAT data. This is getting out of hand.



- Relational databases have the concept of a _view._

"In a database, a view is the result set of a stored query on the data, which the database users can query just as they would in a persistent database collection object. This pre-established query command is kept in the database dictionary. ... Changes applied to the data in a relevant underlying table are reflected in the data shown in subsequent invocations of the view." (https://en.wikipedia.org/wiki/View_(SQL))


- Creating the view

```
CREATE VIEW PlayerBattingYearView
	As SELECT 
	playerID, 
    (select nameLast from Master where Master.playerID = Batting.playerID) as last_name,
    (select nameFirst from Master where Master.playerID = Batting.playerID) as first_name,
    (select franchName from TeamsFranchises WHERE
		TeamsFranchises.franchID = 
			(select franchID from Teams where Teams.teamID = 
                Batting.teamID and Teams.yearID = Batting.yearID)) as franchise_name,
    teamID, yearID, round((h/ab),3) as batting_avg FROM
    lahman2016.batting where ab > 100
    order by batting_avg DESC;
```

- Querying the view


In [12]:
q = "SELECT last_name, first_name, yearID, batting_avg FROM " + \
    " lahman2016.playerbattingyearview where last_name='Williams';"
run_and_print_q("Good hitters named Williams", q)

Execution query = 
 SELECT last_name, first_name, yearID, batting_avg FROM  lahman2016.playerbattingyearview where last_name='Williams';

 Good hitters named Williams 
     last_name first_name  yearID  batting_avg
0    Williams        Ted    1941        0.406
1    Williams        Ted    1957        0.388
2    Williams        Ted    1948        0.369
3    Williams        Ken    1923        0.357
4    Williams        Ted    1942        0.356
5    Williams        Ted    1955        0.356
6    Williams      Jimmy    1899        0.355
7    Williams        Ken    1921        0.347
8    Williams         Cy    1926        0.345
9    Williams        Ken    1929        0.345
10   Williams        Ted    1954        0.345
11   Williams        Ted    1956        0.345
12   Williams        Ted    1940        0.344
13   Williams        Ted    1947        0.343
14   Williams        Ted    1949        0.343
15   Williams     Bernie    1999        0.342
16   Williams        Ted    1946        0.342
17 

- We will go into views in more detail in the next lecture, but I am going to give you a view for HW2 and want to introduce the concept.

## An Interlude on Indexes

BTW, the query

```
SELECT 
    playerID, 
    (select nameLast from Master where Master.playerID = Batting.playerID) as last_name,
    (select nameFirst from Master where Master.playerID = Batting.playerID) as first_name,
    (select franchName from TeamsFranchises WHERE
        TeamsFranchises.franchID = 
                (select franchID from Teams where Teams.teamID = 
                    Batting.teamID and Teams.yearID = Batting.yearID))
                        as franchise_name,
    teamID, yearID, round((h/ab),3) as batting_avg FROM
    lahman2016.batting where ab > 100
    order by batting_avg DESC;
```

runs for 1.5 seconds if there is an index on TeamsFranchises.franchID and 85 seconds if there is no index.



## Homework 2

### Part 1 -- Conceptual Data Model

| <img src="../images/L5_HW_2_CM.jpeg"> | 
|:--:| 
|__HW 2 Conceptual Model__ |


### Logical and Physical Data Model

1. Download and install the full Lahman's 2016 Baseball Database SQL Version. The CAs and I will provide instructions and help.
<BR><BR>
1. HW2 only uses the tables above. Do not worry about the other tables for now.
<br><br>
1. Determine a _primary key_ for each table.
    1. Reminder
        1. A _super key_ is a combination of columns that uniquely identifies a row in a table.
        2. A _candidate key_ is a minimal super key.
        3. A _primary key_ is one of the candidate keys chosen by the data designer/modeler.
    1. I showed you how to determine candidate keys by trying queries. (Lecture 4)
    1. Modify the tables to add a _primary key._ You can use MySQL Workbench or some other tool. 
<br><br>
1. Determine a _foreign key_ relationship for each of the relationships in the diagram above. Create the foreign keys using MySQL Workbench or another tool. The foreign keys will be
    1. There will be two foreign keys in Appearances, Batting, Pitching, Fielding and Salaries.
    1. For each table, one foreign key references Master and one foreign key references Teams.


### Data Access Service API

#### API

- You will write a data access service API. For now, this will be a single function.

```
def dataservice_get(path,fields,filter):
    result = {}
    # Your code goes here.
    return result
```


- path can be of the form
    - /resource
    - /resource/metadata
    - /resource/primary-key
    - /resource/primary-key/resource
    
    
    
- If the path is of the form /resource/primary-key, /resource/primary-key/resource
    - _filter_ will be None
    - _fields_ is a list of fields to get from the identified resources.
    - The function returns a JSON object containing the identified resources and the selected properties.
    
    
    
- If path is of the form /resource
    - _filter_ is a SQL WHERE clause.
    - _fields_ is a list of columns to retrieve.
    - The function returns a JSON object containing the selected resources and the selected properties.
    
    
    
- If path is of the form /resource/metadata, the query returns a JSON object containing the information about the resources columns and keys.



- If the path contains a primary-key, the implementation uses "SHOW KEYS FROM resource" statement to
    - Convert the key in the path of the form k1-k2-k3
    - To the correct WHERE clause.



#### Examples

- Reource query
```
dataservice_get("/master",["nameLast","nameFirst","birthCountry"],"nameLast='Willis'")
```
returns
```
[
  {
      "nameLast": "Willis",
      "nameFirst": "Carl",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Dale",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Dontrelle",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Jim",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Joe",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Lefty",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Les",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Mike",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Ron",
      "birthCountry": "USA"
   },
   {
      "nameLast": "Willis",
      "nameFirst": "Vic",
      "birthCountry": "USA"
   }
]
```

- /resource/primary-key: dataservice_get('/Batting/willite01-1940-1',[ '*' ] , None)
 

In [64]:
d=run_query_to_d("SHOW KEYS FROM Batting WHERE Key_name='PRIMARY'")
print("Batting keys = \n",json.dumps(d,indent=3))

Batting keys = 
 [
   {
      "Table": "batting",
      "Non_unique": 0,
      "Key_name": "PRIMARY",
      "Seq_in_index": 1,
      "Column_name": "playerID",
      "Collation": "A",
      "Cardinality": 18603,
      "Sub_part": null,
      "Packed": null,
      "Null": "",
      "Index_type": "BTREE",
      "Comment": "",
      "Index_comment": ""
   },
   {
      "Table": "batting",
      "Non_unique": 0,
      "Key_name": "PRIMARY",
      "Seq_in_index": 2,
      "Column_name": "yearID",
      "Collation": "A",
      "Cardinality": 93857,
      "Sub_part": null,
      "Packed": null,
      "Null": "",
      "Index_type": "BTREE",
      "Comment": "",
      "Index_comment": ""
   },
   {
      "Table": "batting",
      "Non_unique": 0,
      "Key_name": "PRIMARY",
      "Seq_in_index": 3,
      "Column_name": "stint",
      "Collation": "A",
      "Cardinality": 102492,
      "Sub_part": null,
      "Packed": null,
      "Null": "",
      "Index_type": "BTREE",
      "Comment": "",


    - Defines the key of Batting to be of the form playerID-yearID-stint
    
    - Which means the path /Batting/willite01-1940-1
    
    - Maps to the SQL Query
    
```
SELECT * FROM batting where playerid='willite01' AND yearID='1940' AND stint='1'
```

- The function call dataservice_get(/Master/willite01/Batting,['yearID','ab'],None) maps to the SQL statement
```
SELECT yearID, ab FROM Batting where playerID='willite01'
```


- The function call dataservice_get('/Master/metadata',None,None) returns

```
{
   "fields": [
      {
         "Field": "playerID",
         "Type": "varchar(255)",
         "Null": "NO",
         "Key": "PRI",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "yearID",
         "Type": "int(11)",
         "Null": "NO",
         "Key": "PRI",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "stint",
         "Type": "int(11)",
         "Null": "NO",
         "Key": "PRI",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "teamID",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "lgID",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "G",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "AB",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "R",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "H",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "2B",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "3B",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "HR",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "RBI",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "SB",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "CS",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "BB",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "SO",
         "Type": "int(11)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "IBB",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "HBP",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "SH",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "SF",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      },
      {
         "Field": "GIDP",
         "Type": "varchar(255)",
         "Null": "YES",
         "Key": "",
         "Default": null,
         "Extra": ""
      }
   ],
   "keys": [
      {
         "Table": "batting",
         "Non_unique": 0,
         "Key_name": "PRIMARY",
         "Seq_in_index": 1,
         "Column_name": "playerID",
         "Collation": "A",
         "Cardinality": 18603,
         "Sub_part": null,
         "Packed": null,
         "Null": "",
         "Index_type": "BTREE",
         "Comment": "",
         "Index_comment": ""
      },
      {
         "Table": "batting",
         "Non_unique": 0,
         "Key_name": "PRIMARY",
         "Seq_in_index": 2,
         "Column_name": "yearID",
         "Collation": "A",
         "Cardinality": 93857,
         "Sub_part": null,
         "Packed": null,
         "Null": "",
         "Index_type": "BTREE",
         "Comment": "",
         "Index_comment": ""
      },
      {
         "Table": "batting",
         "Non_unique": 0,
         "Key_name": "PRIMARY",
         "Seq_in_index": 3,
         "Column_name": "stint",
         "Collation": "A",
         "Cardinality": 102492,
         "Sub_part": null,
         "Packed": null,
         "Null": "",
         "Index_type": "BTREE",
         "Comment": "",
         "Index_comment": ""
      }
   ]
}
```
